In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # or the full path to the "project" directory
import numpy as np
import gpbr

In [2]:
T = 2 # final time
N = 9 # N+1=10 time points
# M = 64 # number of collocation points
# M = 16 # number of collocation points
M = 16 # number of collocation points
M1= 4
M2= 8
M=M1*M2 # 

In [3]:
M

32

In [4]:
tn = np.array([(n+1)*(T/(N+1)) for n in range(0, N+1)])
h = T/(N+1)

In [5]:
nu = np.sqrt(2/h)
nu

np.float64(3.1622776601683795)

In [6]:
betta_array = []
for n in range(0, N+1):
    sign = (-1)**n
    betta_array.append(sign*(4/h))
betta_array[0] = np.nan
betta_array

[nan, -20.0, 20.0, -20.0, 20.0, -20.0, 20.0, -20.0, 20.0, -20.0]

In [7]:
from gpbr.direct.heat_equation.common import MFSData
mfs_data =MFSData(N, T, tn, M, betta_array, nu)

In [8]:
from gpbr.direct.common.collocation import collocation_points_3d, CollocationData3D
coll_3d = collocation_points_3d(M1,M2)
coll_3d

CollocationData3D(n_theta=4, n_phi=8, theta=array([0.62831853, 1.25663706, 1.88495559, 2.51327412]), phi=array([0.78539816, 1.57079633, 2.35619449, 3.14159265, 3.92699082,
       4.71238898, 5.49778714, 6.28318531]), theta_grid=array([[0.62831853, 1.25663706, 1.88495559, 2.51327412],
       [0.62831853, 1.25663706, 1.88495559, 2.51327412],
       [0.62831853, 1.25663706, 1.88495559, 2.51327412],
       [0.62831853, 1.25663706, 1.88495559, 2.51327412],
       [0.62831853, 1.25663706, 1.88495559, 2.51327412],
       [0.62831853, 1.25663706, 1.88495559, 2.51327412],
       [0.62831853, 1.25663706, 1.88495559, 2.51327412],
       [0.62831853, 1.25663706, 1.88495559, 2.51327412]]), phi_grid=array([[0.78539816, 0.78539816, 0.78539816, 0.78539816],
       [1.57079633, 1.57079633, 1.57079633, 1.57079633],
       [2.35619449, 2.35619449, 2.35619449, 2.35619449],
       [3.14159265, 3.14159265, 3.14159265, 3.14159265],
       [3.92699082, 3.92699082, 3.92699082, 3.92699082],
       [4.71238898, 

In [9]:
from gpbr.direct.common.boundary import StarlikeSurface, starlike_sphere_base
point_sphere = starlike_sphere_base(coll_3d)

Gamma1 = point_sphere*0.5
Gamma2 = point_sphere*1.5

In [10]:
from gpbr.direct.common.source import source_points_3d, SourcePoints3D

source_coll_3d = collocation_points_3d(M1, M2)
source_point_sphere =  starlike_sphere_base(source_coll_3d)
eta1 = 0.5
eta2 = 2.0

Gamma1_source = source_point_sphere*0.5
Gamma2_source = source_point_sphere*1.5

source_points = source_points_3d(eta1, eta2, Gamma1_source, Gamma2_source)

# TODO: Add graph

In [11]:
from gpbr.direct.common.boundary import Point2D, Point3D
def f1(x: Point2D | Point3D , t: float):
    return 0

# def f2(x: Point2D , t: float) -> np.array:
#     return (t**2)*np.exp(-4*t+2)*np.sqrt(x.x**2 + x.y**2)
def f2(x: Point2D , t: float) -> np.array:
    return 1.0

In [12]:
from gpbr.direct.heat_equation.polynomial import MFSPolinomials3D, calculate_3d_polinomials

mfs_polynomyals = calculate_3d_polinomials(mfs_data, N)

In [13]:
from gpbr.direct.heat_equation.fundamental_sequence import fundamental_sequence_3d

fundamental_sequence_gamma1 = fundamental_sequence_3d(Gamma1, source_points, mfs_data, mfs_polynomyals)
fundamental_sequence_gamma2 = fundamental_sequence_3d(Gamma2, source_points, mfs_data, mfs_polynomyals)

In [14]:
from gpbr.direct.heat_equation.helpers import form_fs_matrix, form_fs_vector_3d
PHI_MAT = form_fs_matrix(fundamental_sequence_gamma1, fundamental_sequence_gamma2)
PHI_MAT

array([[1.47455388e-04, 9.90129103e-05, 3.78254608e-05, ...,
        7.14021113e-05, 2.81898879e-05, 1.10590907e-05],
       [9.90129103e-05, 1.47455388e-04, 9.90129103e-05, ...,
        8.52896279e-05, 5.86927077e-05, 2.81898879e-05],
       [3.78254608e-05, 9.90129103e-05, 1.47455388e-04, ...,
        5.86927077e-05, 8.52896279e-05, 7.14021113e-05],
       ...,
       [2.68032746e-04, 5.13465125e-04, 1.37051727e-04, ...,
        5.80589442e-03, 9.21860701e-04, 3.46285844e-05],
       [1.48192523e-05, 1.37051727e-04, 5.13465125e-04, ...,
        9.21860701e-04, 5.80589442e-03, 9.21860701e-04],
       [1.27090261e-06, 1.48192523e-05, 2.68032746e-04, ...,
        3.46285844e-05, 9.21860701e-04, 5.80589442e-03]])

In [15]:
from gpbr.direct.heat_equation.fundamental_sequence import FundamentalSequenceCoefs

alpha_coeeff = np.empty((N+1, M), dtype=np.float64)
alpha_coeeff[:] = np.nan
fs_coefs = FundamentalSequenceCoefs(alpha_coeeff)

In [20]:
for n in range(0, N+1):
    F = form_fs_vector_3d(n,
                          fundamental_sequence_gamma1,
                          fundamental_sequence_gamma2,
                          Gamma1,
                          Gamma2,
                          source_points,
                          fs_coefs,
                          f1, f2, mfs_data, mfs_polynomyals)
    # print(F)
    from numpy.linalg import lstsq
    alpha_n = lstsq(PHI_MAT, F)[0]
    fs_coefs.alpha[n] = alpha_n.T
    print(fs_coefs.alpha)

[[ 7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01
   7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01
   7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01
   7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01
   7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01
   7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01
   7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01
   7.72215769e+01  1.07083716e+02  1.07083716e+02  7.72215769e+01]
 [-4.33458345e+02 -5.95421241e+02 -5.95421241e+02 -4.33458345e+02
  -4.33458345e+02 -5.95421241e+02 -5.95421241e+02 -4.33458345e+02
  -4.33458345e+02 -5.95421241e+02 -5.95421241e+02 -4.33458345e+02
  -4.33458345e+02 -5.95421241e+02 -5.95421241e+02 -4.33458345e+02
  -4.33458345e+02 -5.95421241e+02 -5.95421241e+02 -4.33458345e+02
  -4.33458345e+02 -5.95421241e+02 -5.95421241e+02 -4.33458345e+02
  -4.33458345e+02 -5.95421241e+02 -5.95421241e+02 -4.33458345e+02
  -4.3345

In [17]:
from gpbr.direct.heat_equation.helpers import u_3d
u_3d(Point3D(1,1,1), 1, source_points, fs_coefs, mfs_polynomyals, mfs_data)

m = 0
m = 1


np.float64(-65.35544707476265)